In [1]:
# Voila Dashboard Launcher
# Save as app_dashboard.ipynb in App_dashboard folder

import ipywidgets as widgets
from IPython.display import display, HTML, IFrame
import os
import sys
import glob
import re
sys.path.append(os.path.dirname(os.getcwd()))
import access_token

access_token.log_notebook_usage()

# ======================== CONFIGURATION SECTION ========================
# Based on the user's complete URL example

# The complete base URL path including user-specific elements
# From: https://nomad-hzb-se.de/nomad-oasis/north/user/edgar/voila/voila/render/uploads/...
FULL_VOILA_PREFIX = "/nomad-oasis/north/user/edgar/voila/voila/render"

# The URL path without the user-specific part (for documentation)
GENERIC_VOILA_PREFIX = "/nomad-oasis/north/user/[USERNAME]/voila/voila/render"

# Path to uploads folder
UPLOADS_PATH = "uploads/analysis-tools-mr60amaQRZ-Ta21fXdf64Q"

# Option to enable/disable path checking (set to False if path checking causes errors)
CHECK_PATHS = True

# ======================== HELPER FUNCTIONS ========================

def check_path_exists(path):
    """Verify if a notebook path exists and return a friendlier error if not"""
    if not CHECK_PATHS:
        return True, None
    
    if not os.path.exists(path):
        return False, f"Warning: Could not find notebook at '{path}'. Check the path and notebook name."
    return True, None

def try_various_paths(app_folder, notebook_name):
    """Try different path combinations to help identify the correct one"""
    possible_paths = [
        f"../{app_folder}/{notebook_name}",
        f"../../{app_folder}/{notebook_name}",
        f"{app_folder}/{notebook_name}",
    ]
    
    print(f"Trying possible paths for {app_folder}/{notebook_name}:")
    for path in possible_paths:
        exists = os.path.exists(path)
        print(f" - {path}: {'EXISTS' if exists else 'Not found'}")
    
    return

def on_app_button_clicked(b):
    """Handle app button click by launching the selected notebook in Voila"""
    app_folder = b.app_folder
    notebook_name = b.notebook_name
    
    # Path to check if the file exists locally
    local_path = f"../{app_folder}/{notebook_name}"
    
    # Different URL formats to try - now with the proper user path prefix
    voila_url = f"{FULL_VOILA_PREFIX}/{UPLOADS_PATH}/{app_folder}/{notebook_name}"
    jupyter_url = f"/notebooks/{UPLOADS_PATH}/{app_folder}/{notebook_name}"
    
    # Check if the path exists locally
    exists, error_msg = check_path_exists(local_path)
    if not exists and CHECK_PATHS:
        print(error_msg)
        display(HTML(f"""
            <div style="color: #721c24; background-color: #f8d7da; padding: 10px; 
                 border: 1px solid #f5c6cb; border-radius: 5px; margin: 10px 0;">
                {error_msg}<br>
                Attempted path: {local_path}<br>
                Please check that the app folder and notebook name are correct.
            </div>
        """))
    
    # Create a more comprehensive launch panel with multiple options
    display(HTML(f"""
        <div style="margin: 10px 0; padding: 15px; background-color: #d4edda; border-radius: 5px; border-left: 5px solid #28a745;">
            <h4 style="margin-top: 0;">Launch {b.description}</h4>
            
            <div style="display: flex; flex-direction: column; gap: 10px; margin: 15px 0;">
                <div>
                    <strong>Option 1 (Voila with user path):</strong><br>
                    <a href="{voila_url}" target="_blank" style="display: inline-block; padding: 8px 15px; background-color: #007bff; 
                      color: white; text-decoration: none; border-radius: 5px; margin-top: 5px;">
                      Open in Voila
                    </a>
                    <div style="margin-top: 3px;"><small>URL: <code>{voila_url}</code></small></div>
                </div>
                
                <div>
                    <strong>Option 2 (Open in Jupyter):</strong><br>
                    <a href="{jupyter_url}" target="_blank" style="display: inline-block; padding: 8px 15px; background-color: #fd7e14; 
                      color: white; text-decoration: none; border-radius: 5px; margin-top: 5px;">
                      Open in Jupyter
                    </a>
                    <div style="margin-top: 3px;"><small>Opens the notebook directly in the Jupyter interface</small></div>
                </div>
                
                <div>
                    <strong>Option 3 (Manual URL):</strong><br>
                    <div style="margin-top: 3px;">
                        <p>If Option 1 doesn't work, copy this URL into your browser:</p>
                        <div style="font-family: monospace; background: #f8f9fa; padding: 5px; border-radius: 3px; word-break: break-all;">
                            https://nomad-hzb-se.de{voila_url}
                        </div>
                    </div>
                </div>
            </div>
            
            <div style="margin-top: 15px; padding-top: 10px; border-top: 1px solid #c3e6cb; font-size: 0.9em;">
                <strong>Note:</strong> These links use your current user path (<code>edgar</code>). If accessed by another user, 
                they will need to replace this with their own username in the URL.
            </div>
        </div>
    """))
    
    # Provide feedback in the current notebook
    print(f"Launching {b.description} - options provided above...")

def on_test_paths_button_clicked(b):
    """Test various path options to debug the environment"""
    print("Testing path discovery...")
    
    # Current working directory
    print(f"Current working directory: {os.getcwd()}")
    
    # Try various relative paths
    for level in range(1, 4):
        path = "../" * level
        try:
            print(f"\nContents of {path}:")
            for item in os.listdir(path):
                if os.path.isdir(os.path.join(path, item)):
                    print(f" - {item}/ (directory)")
                else:
                    print(f" - {item} (file)")
        except Exception as e:
            print(f"  Error listing {path}: {e}")
    
    # Try the paths for the first application
    if applications:
        app_folder, notebook_name = applications[0][0], applications[0][1]
        try_various_paths(app_folder, notebook_name)

def on_detailed_folder_scan(b):
    """Perform a detailed scan of app folders to find notebook files"""
    print("Scanning application folders for notebook files...")
    
    # List of app folders we saw in the parent directory
    app_folders = [
        "MPPT_Analysis", 
        "EQE_Analysis", 
        "JV_Analysis", 
        "Data_uploader",
        "Overview_Dashboard",
        "Perovskite_calculator",
        "Hansen_green_calculator",
        "Diode_Analyzer",
        "Wetting_envelope"
    ]
    
    # Scan each folder for notebook files
    for folder in app_folders:
        folder_path = f"../{folder}"
        try:
            print(f"\nContents of {folder_path}/:")
            files = os.listdir(folder_path)
            notebook_files = [f for f in files if f.endswith('.ipynb')]
            
            if notebook_files:
                print(f"  Found notebook files:")
                for nb in notebook_files:
                    print(f"  - {nb}")
            else:
                print(f"  No notebook files found")
                
            # Look for other potentially useful files
            json_files = [f for f in files if f.endswith('.json')]
            py_files = [f for f in files if f.endswith('.py')]
            
            if json_files or py_files:
                print(f"  Other relevant files:")
                for f in json_files + py_files:
                    print(f"  - {f}")
                    
        except Exception as e:
            print(f"  Error scanning {folder_path}: {e}")
    
    print("\nTo use these notebook files, update the 'applications' list with the correct filenames.")

# Define your available applications with the correct notebook names
# Format: (app_folder, notebook_name, display_name, description, icon_class)
applications = [
    ("MPPT_Analysis", "mppt plotting.ipynb", "MPPT Analyzer", "Analyze maximum power point tracking data for your solar cells", "fa-chart-line"),
    ("EQE_Analysis", "eqe plotter.ipynb", "EQE Analyzer", "Visualize and analyze external quantum efficiency measurements", "fa-chart-area"),
    ("JV_Analysis", "run_analysis.ipynb", "JV Analysis Tool", "Examine current-voltage characteristics of your devices", "fa-chart-bar"),
    ("Data_uploader", "upload_data_for_sample.ipynb", "File Uploader", "Upload measurement files to the NOMAD platform", "fa-upload"),
    ("Overview_Dashboard", "production_dashboard.ipynb", "Samples Overview", "Get a comprehensive view of all your research samples", "fa-table"),
    ("Perovskite_calculator", "PerovskiteCalculator_Standalone(1).ipynb", "Perovskite Calculator", "Calculate perovskite properties and compositions", "fa-calculator"),
    ("Hansen_green_calculator", "hansen_blend_calculator.ipynb", "Hansen Calculator", "Use the Hansen solubility parameter calculator", "fa-flask"),
    ("Diode_Analyzer", "diode-gui.ipynb", "Diode Analyzer", "Analyze diode characteristics of your devices", "fa-microchip")
]

# Optionally define custom images instead of icons (empty string means use the icon instead)
# Format: {folder_name: image_path}
custom_images = {
    # Example: "MPPT_Analysis": "images/mppt_icon.png",
    # Leave empty if you want to use Font Awesome icons for all
}

# ======================== DASHBOARD UI ========================

# Add configuration panel with enhanced information
config_html = f"""
<div style="margin-bottom: 30px; padding: 15px; background-color: #e9f7fe; border-radius: 8px; border-left: 5px solid #3498db;">
    <h3>Dashboard Configuration</h3>
    <div style="display: flex; justify-content: space-between; flex-wrap: wrap;">
        <div style="flex: 1; min-width: 300px;">
            <p><strong>Current Directory:</strong> <code>/home/jovyan/uploads/analysis-tools-mr60amaQRZ-Ta21fXdf64Q/App_dashboard</code></p>
            <p><strong>App Folders:</strong> <code>/home/jovyan/uploads/analysis-tools-mr60amaQRZ-Ta21fXdf64Q/[APP_FOLDER]</code></p>
            <p><strong>Full Voila Path:</strong> <code>{FULL_VOILA_PREFIX}</code></p>
        </div>
        <div style="flex: 1; min-width: 300px; margin-top: 10px;">
            <h4 style="margin-top: 0;">URL Structure for Other Users</h4>
            <p>The dashboard is configured for user <strong>edgar</strong>. Other users should:</p>
            <ol style="margin-top: 5px;">
                <li>Replace <code>edgar</code> with their username in the URL</li>
                <li>Generic pattern: <code>{GENERIC_VOILA_PREFIX}</code></li>
                <li>URLs will link to your NOMAD Voila instance</li>
            </ol>
        </div>
    </div>
</div>
"""
display(HTML(config_html))

# Create buttons
test_paths_button = widgets.Button(
    description="Test Path Discovery",
    button_style='warning',
    layout=widgets.Layout(width='200px', height='40px')
)
test_paths_button.on_click(on_test_paths_button_clicked)

detailed_scan_button = widgets.Button(
    description="Scan App Folders",
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)
detailed_scan_button.on_click(on_detailed_folder_scan)

# Display buttons in a row
button_container = widgets.HBox([test_paths_button, detailed_scan_button])
display(button_container)

# Add a success message and set title
updated_note = """
<div style="margin: 20px 0; padding: 10px; background-color: #d4edda; border-radius: 5px; border-left: 5px solid #28a745;">
    <h4 style="margin-top: 0;">✓ Notebook Names Updated</h4>
    <p>The dashboard has been configured with the correct notebook filenames.</p>
    <p>Click any application button below to generate multiple launch options.</p>
</div>
"""
display(HTML(updated_note))

# Set title and styling
display(HTML("<h1>NOMAD Analysis Tools Dashboard</h1>"))
display(HTML("<p>Select an application to launch from the options below.</p>"))

# Add Font Awesome for icons
display(HTML('<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css">'))

# Configure the style of the dashboard
style = """
<style>
.dashboard-container {
    display: flex;
    flex-direction: column;
    margin: 20px 0;
}
.app-button {
    margin: 10px 0;
    padding: 15px;
    border-radius: 8px;
    background-color: #f5f5f5;
    transition: background-color 0.3s;
    display: flex;
    align-items: center;
}
.app-button:hover {
    background-color: #e0e0e0;
}
.app-icon {
    width: 50px;
    height: 50px;
    font-size: 24px;
    display: flex;
    align-items: center;
    justify-content: center;
    margin-right: 15px;
    color: #3498db;
    background-color: rgba(52, 152, 219, 0.1);
    border-radius: 8px;
}
.app-icon img {
    max-width: 40px;
    max-height: 40px;
}
.app-button-container {
    display: flex;
    align-items: center;
}
.app-description {
    margin-left: 20px;
    color: #555;
}
</style>
"""
display(HTML(style))

# Create a container for all app buttons
container = widgets.VBox([])
app_buttons = []

# Create buttons for each application
for app_folder, notebook_name, display_name, description, icon_class in applications:
    # Create a button for this app
    button = widgets.Button(
        description=display_name,
        button_style='info',
        layout=widgets.Layout(width='300px', height='50px')
    )
    
    # Store the notebook and folder path as attributes on the button
    button.notebook_name = notebook_name
    button.app_folder = app_folder
    
    # Attach the click handler
    button.on_click(on_app_button_clicked)
    
    # Create icon or image element
    if app_folder in custom_images and custom_images[app_folder]:
        # Use custom image
        icon_html = f"<div class='app-icon'><img src='{custom_images[app_folder]}' alt='{display_name}'></div>"
    else:
        # Use Font Awesome icon
        icon_html = f"<div class='app-icon'><i class='fas {icon_class}'></i></div>"
    
    icon_widget = widgets.HTML(value=icon_html)
    
    # Create a description label
    desc_label = widgets.HTML(value=f"<div class='app-description'>{description}</div>")
    
    # Create a container for the button and icon
    button_container = widgets.HBox([icon_widget, button])
    button_container.add_class("app-button-container")
    
    # Create a container for this app button and description
    app_container = widgets.HBox([button_container, desc_label])
    app_container.add_class("app-button")
    
    # Add to our list of buttons
    app_buttons.append(app_container)

# Add all buttons to the container
container.children = app_buttons
display(container)

# Add footer with instructions
footer = """
<div style="margin-top: 30px; padding: 20px; background-color: #f9f9f9; border-radius: 8px;">
    <h3>Instructions:</h3>
    <ul>
        <li>Click on any application button above to launch it in a new tab</li>
        <li>All applications are powered by Voila and your data in NOMAD</li>
        <li>Return to this dashboard to launch different tools as needed</li>
    </ul>
    <p><em>Contact the data team for help or to request new analysis tools</em></p>
    
    <div style="margin-top: 15px; padding-top: 15px; border-top: 1px solid #ddd;">
        <h4>Customizing the Dashboard:</h4>
        <p>To add or modify applications:</p>
        <ol>
            <li>Update the <code>applications</code> list with the correct folder and notebook names</li>
            <li>Make sure the folder names match exactly what's in the file system</li>
            <li>Notebook names should include the <code>.ipynb</code> extension</li>
        </ol>
    </div>
    
    <div style="margin-top: 15px; padding-top: 15px; border-top: 1px solid #ddd;">
        <h4>Adding Custom App Images:</h4>
        <p>To add your own images for each application:</p>
        <ol>
            <li>Create an <code>images</code> folder in the same directory as this dashboard</li>
            <li>Add your image files (PNG or JPG recommended, 40x40px ideal size)</li>
            <li>Update the <code>custom_images</code> dictionary in this notebook</li>
        </ol>
    </div>
</div>
"""
display(HTML(footer))